In [17]:
import pydantic


In [1]:
class User:
    def __init__(self,id: int,name="akif agayev"):
        if not isinstance(id,int):
            raise TypeError(f"Expected id tobe an int {type(id).__name__}")
        
        if not isinstance(name,str):
            raise TypeError(f"Expected name tobe an str {type(name).__name__}")
        

        self.id = id
        self.name = name

try:
    user = User("1254")   
except TypeError as e:
    print(e)





Expected id tobe an int str


In [2]:
from pydantic import BaseModel


class User(BaseModel):

    id : int
    name : str = "malik hesenov"


user = User(id="45") 
print(user)   
print(user.model_fields_set)
user = User(id=45,name="hafiz haciyev")
print(user.model_fields_set)

id=45 name='malik hesenov'
{'id'}
{'name', 'id'}


In [3]:
print(user.model_dump())
print(user.model_dump_json())
print(user.model_json_schema())

{'id': 45, 'name': 'hafiz haciyev'}
{"id":45,"name":"hafiz haciyev"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'malik hesenov', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


In [4]:
from typing import Optional,List


class Food(BaseModel):
    name : str
    price : float
    ingridients : Optional[List[str]] = None


class Resturant(BaseModel):
    name: str
    location: str
    foods: List[Food] 


resturant_instance = Resturant(
    name="abseron mariot",
    location="Baki",
    foods=[
        {'name':"Cheese pizza","price":45,"ingridients":["Cheese","tomato","Dough"]},
        {'name':"Rice","price":852,"ingridients":["rice","poteto","Dough"]},
        {'name':"Rice-souse","price":552}
    ]
)       


print(resturant_instance)
print(resturant_instance.model_dump())


name='abseron mariot' location='Baki' foods=[Food(name='Cheese pizza', price=45.0, ingridients=['Cheese', 'tomato', 'Dough']), Food(name='Rice', price=852.0, ingridients=['rice', 'poteto', 'Dough']), Food(name='Rice-souse', price=552.0, ingridients=None)]
{'name': 'abseron mariot', 'location': 'Baki', 'foods': [{'name': 'Cheese pizza', 'price': 45.0, 'ingridients': ['Cheese', 'tomato', 'Dough']}, {'name': 'Rice', 'price': 852.0, 'ingridients': ['rice', 'poteto', 'Dough']}, {'name': 'Rice-souse', 'price': 552.0, 'ingridients': None}]}


In [5]:
!pip install pydantic[email]



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from typing import List
from pydantic import BaseModel,EmailStr,PositiveInt,conlist,Field,HttpUrl
import re

class Address(BaseModel):
    street : str
    city : str
    state : str
    zip_code : str


class Employee(BaseModel):
    name : str
    position: str
    email : EmailStr

class Owner(BaseModel):
    name: str
    email: EmailStr



class Resturant(BaseModel):
    name :str = Field(...,pattern=r"^[a-zA-Z0-9]+$")
    owner : Owner
    address : Address
    employees : conlist(Employee ,min_length=2)
    number_of_seats : PositiveInt
    delivery : bool
    website : HttpUrl


resturant_instance_1 = Resturant(
    name="Tastyparty",
    owner={
        "name":"eldar qasimov",
        "email":"eldar@mail.ru"
    },
    address={
        "street":"h.cavid",
        "city":"baku",
        "state":"class",
        "zip_code":"456"
    },
    employees=[
        {"name":"david collins",
        "position":"servant",
        "email":"collins@gmail.com"},

        {"name":"david bakhem",
        "position":"CEO",
        "email":"beckhams@gmail.com"}
        ],
    number_of_seats= 45,
    delivery=True,
    website="https://www.website.com"
        )    


In [7]:
from pydantic import BaseModel,EmailStr,field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls,v:str) -> str:

        if " " not in v:
            raise ValueError("Owner name must contain a space")
        return v.upper()
    

try:
    owner_instance = Owner(name="el dar",email="eldar@gmail.com") 
except ValueError as e:
    print(e)

print(owner_instance.name)      
    


EL DAR


In [8]:
from typing import Any
from pydantic import BaseModel,EmailStr,ValidationError,model_validator


class Owner(BaseModel):
    name:str
    email: EmailStr


    @model_validator(mode="before")
    @classmethod
    def check_sensitive_info_omitted(cls,data:Any) -> Any:
        if isinstance(data,dict):
            if "password" in data:
                raise ValueError("password should not be included")
            if "card_number" in data:
                raise ValueError("card number should not be included")
        return data

    @model_validator(mode="after")
    def check_name_contains_space(self) -> 'Owner':

        if  " " not in self.name:
            raise ValueError("Owner name must contain space")
        return self


print(Owner(name="Ucal Efendi",email="ucal@mail.ru"))    


try:
    # owner=Owner(name="Ucal Efendi",email="ucal@mail.ru",password="12345")
    owner=Owner(name="Ucal Efendi",email="ucal@mail.ru")
except ValidationError as e:
        print(e)


name='Ucal Efendi' email='ucal@mail.ru'


In [9]:
##
from pydantic import BaseModel,Field


class User(BaseModel):

    name :str = "Eldar Memmedov"

    name :str = Field(default="Ucal Efendi")


user = User()

print(user.name)


Ucal Efendi


In [10]:
from uuid import uuid4

from pydantic import BaseModel,Field

class User(BaseModel):

    id :int = Field(default_factory=lambda : uuid4().hex)


user = User()

print(user)

id='a151c67a22f3423bbbac87cb0760a3f6'


In [11]:
from pydantic import BaseModel,Field

class User(BaseModel):

    name :str = Field(...,alias="username")



user = User(username="ayten salmanov")
print(user)
print(user.model_dump(by_alias=True))
print(type(user.model_dump(by_alias=True)))

name='ayten salmanov'
{'username': 'ayten salmanov'}
<class 'dict'>


In [12]:
#look at Field constraints


from pydantic import BaseModel,Field,EmailStr
from typing import List
from decimal import Decimal


class User(BaseModel):
    username :str = Field(...,min_length=3,max_length=10,pattern=r'^\w+$')
    email : str = Field(...)
    age :int = Field(...,gt=0,le=120)
    heigh :float = Field(...,gt=0.0)
    is_active :bool  = Field(True)
    balance :Decimal = Field(...,max_digits=10,decimal_places=2)
    favorite_number :List[int] = Field(...,min_items=1)


user = User(
    username="564",
    email="emailabdul@mail.ru",
    age=5,
    heigh=56,
    is_active=True,
    balance=6,
    favorite_number=[45,85,87,8,25,154]
)    

print(user.model_dump())


{'username': '564', 'email': 'emailabdul@mail.ru', 'age': 5, 'heigh': 56.0, 'is_active': True, 'balance': Decimal('6'), 'favorite_number': [45, 85, 87, 8, 25, 154]}


In [13]:
from pydantic import BaseModel,computed_field
from datetime import datetime


class Person(BaseModel):
    name: str
    birth_year: int

    @computed_field
    @property
    def age(self) ->int:

        current_year = datetime.now().year
        return current_year - self.birth_year
    

print(Person(name="Ucal",birth_year=1994).model_dump()) 

print(Person.age)


{'name': 'Ucal', 'birth_year': 1994, 'age': 31}


In [14]:
from pydantic import BaseModel,ValidationError,field_validator
from datetime import datetime



class Person(BaseModel):
    name: str
    birth_year: int

    @computed_field
    @property
    def age(self) ->int:

        current_year = datetime.now().year
        return current_year - self.birth_year
    
    @field_validator("birth_year")
    @classmethod
    def validate_age(cls, v :int) -> int:

        current_year = datetime.now().year
        if current_year - v < 18:
            ValidationError("Person must be 18 years or older")

        return v 


try:
    print(Person(name="Ucal Efendi",birth_year=455))   
except ValidationError as e:
    print(e)  
     


name='Ucal Efendi' birth_year=455 age=1570


In [15]:
from dataclasses import dataclass ,field
from typing import List,Optional
from pydantic import Field,TypeAdapter

@dataclass
class User:
    id :int
    name: str = "ucal efendi"
    age: Optional[int] = field(default=None,metadata=dict(title="The age of user",description = "do not lie",ge = 18))
    height: Optional[int] = Field(None,title="the height in cm",ge=50,le=300) 
    friends: List[int]  = field(default_factory= lambda : [0])  



print(len(TypeAdapter(User).json_schema()))

    

4


In [16]:
from pydantic import BaseModel,ValidationError



class User(BaseModel):
    id :int
    username :str


print(User.model_validate({"id":1,'username':"halkman"}))    

id=1 username='halkman'


In [20]:
try:
    User.model_validate({"id":5,"username":855},strict=True)
except ValidationError as exc:
    print(exc)  


1 validation error for User
username
  Input should be a valid string [type=string_type, input_value=855, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


In [38]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings


class Settings(BaseSettings):
    auth_key: str = Field(...)
    api_key: str = Field(...,alias="my_api_key")


    class Config:
        populate_by_name = True







print(Settings(auth_key="test_auth_key",my_api_key="test_api_key").model_dump())    

{'auth_key': 'test_auth_key', 'api_key': 'test_api_key'}


In [ ]:
import os
from pydantic import Field,AliasChoices
from pydantic_settings import BaseSettings

os.environ["AUTH_KEY"] = "test_auth_key"
os.environ['MY_API_KEY'] = "test"
os.environ["ENV2"] = "http://mysuperurl.com"


class Setting(BaseSettings):

    service_name:str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias="my_api_key")
    url: str = Field(validation_alias=AliasChoices("env1","env2"))